# Advanced indexing

In [1]:
import sys
sys.path.insert(0, '..')
import zarr
import numpy as np
np.random.seed(42)
import cProfile
zarr.__version__

'2.1.5.dev113'

## Functionality and API

### Indexing a 1D array with a Boolean array

Supported via ``__getitem__`` and ``__setitem__`` just like numpy array.

In [3]:
a = np.arange(10)
za = zarr.array(a, chunks=2)
ix = [False,  True,  False,  True, False, True, False,  True,  False,  True]

In [4]:
# get items
za[ix]

array([1, 3, 5, 7, 9])

In [5]:
# set items
za[ix] = a[ix] * 10
za[:]

array([ 0, 10,  2, 30,  4, 50,  6, 70,  8, 90])

In [6]:
# indexing array can be any array-like, e.g., Zarr array
zix = zarr.array(ix, chunks=2)
za = zarr.array(a, chunks=2)
za[zix]  # will not load all zix into memory

array([1, 3, 5, 7, 9])

### Indexing a 1D array with an integer array

Supported via ``__getitem__`` and ``__setitem__`` just like numpy array.

In [7]:
a = np.arange(10)
za = zarr.array(a, chunks=2)
ix = [1, 3, 5, 7, 9]

In [8]:
# get items
za[ix]

array([1, 3, 5, 7, 9])

In [9]:
# set items
za[ix] = a[ix] * 10
za[:]

array([ 0, 10,  2, 30,  4, 50,  6, 70,  8, 90])

### Slicing a 1D array with step > 1

Slices with step > 1 are supported. Internally these are converted to an integer array via ``np.arange``.

In [10]:
a = np.arange(10)
za = zarr.array(a, chunks=2)

In [11]:
# get items
za[1::2]

array([1, 3, 5, 7, 9])

In [12]:
# set items
za[1::2] = a[1::2] * 10
za[:]

array([ 0, 10,  2, 30,  4, 50,  6, 70,  8, 90])

### Orthogonal (outer) indexing of multi-dimensional arrays

Orthogonal (a.k.a. outer) indexing is supported with either Boolean or integer arrays. This functionality is provided via the ``get/set_orthogonal_selection()`` methods. For convenience, this functionality is also available via the ``oindex[]`` property as has been proposed for numpy.

In [13]:
a = np.arange(15).reshape(5, 3)
za = zarr.array(a, chunks=(3, 2))
za[:]

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [14]:
# orthogonal indexing with Boolean arrays
ix0 = [False, True, False, True, False]
ix1 = [True, False, True]
za.get_orthogonal_selection((ix0, ix1))

array([[ 3,  5],
       [ 9, 11]])

In [15]:
# alternative API
za.oindex[ix0, ix1]

array([[ 3,  5],
       [ 9, 11]])

In [16]:
# orthogonal indexing with integer arrays
ix0 = [1, 3]
ix1 = [0, 2]
za.get_orthogonal_selection((ix0, ix1))

array([[ 3,  5],
       [ 9, 11]])

In [17]:
# alternative API
za.oindex[ix0, ix1]

array([[ 3,  5],
       [ 9, 11]])

In [18]:
# combine with slice
za.oindex[[1,  3], :]

array([[ 3,  4,  5],
       [ 9, 10, 11]])

In [19]:
# combine with slice
za.oindex[:, [0, 2]]

array([[ 0,  2],
       [ 3,  5],
       [ 6,  8],
       [ 9, 11],
       [12, 14]])

In [20]:
# set items via Boolean selection
ix0 = [False, True, False, True, False]
ix1 = [True, False, True]
selection = ix0, ix1
value = 42
za.set_orthogonal_selection(selection, value)
za[:]

array([[ 0,  1,  2],
       [42,  4, 42],
       [ 6,  7,  8],
       [42, 10, 42],
       [12, 13, 14]])

In [21]:
# alternative API
za.oindex[ix0, ix1] = 44
za[:]

array([[ 0,  1,  2],
       [44,  4, 44],
       [ 6,  7,  8],
       [44, 10, 44],
       [12, 13, 14]])

In [22]:
# set items via integer selection
ix0 = [1, 3]
ix1 = [0, 2]
selection = ix0, ix1
value = 46
za.set_orthogonal_selection(selection, value)
za[:]

array([[ 0,  1,  2],
       [46,  4, 46],
       [ 6,  7,  8],
       [46, 10, 46],
       [12, 13, 14]])

In [23]:
# alternative API
za.oindex[ix0, ix1] = 48
za[:]

array([[ 0,  1,  2],
       [48,  4, 48],
       [ 6,  7,  8],
       [48, 10, 48],
       [12, 13, 14]])

### Coordinate indexing of multi-dimensional arrays

Selecting arbitrary points from a multi-dimensional array by indexing with integer (coordinate) arrays is supported. This functionality is provided via the ``get/set_coordinate_selection()`` methods. For convenience, this functionality is also available via the ``vindex[]`` property as has been proposed for numpy.

In [24]:
a = np.arange(15).reshape(5, 3)
za = zarr.array(a, chunks=(3, 2))
za[:]

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [25]:
# get items
ix0 = [1, 3]
ix1 = [0, 2]
za.get_coordinate_selection((ix0, ix1))

array([ 3, 11])

In [26]:
# alternative API
za.vindex[ix0, ix1]

array([ 3, 11])

In [27]:
# set items
za.set_coordinate_selection((ix0, ix1), 42)
za[:]

array([[ 0,  1,  2],
       [42,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 42],
       [12, 13, 14]])

In [28]:
# alternative API
za.vindex[ix0, ix1] = 44
za[:]

array([[ 0,  1,  2],
       [44,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 44],
       [12, 13, 14]])

### Mask indexing of multi-dimensional arrays

Selecting arbitrary points from a multi-dimensional array by a Boolean array is supported. This functionality is provided via the ``get/set_mask_selection()`` methods. For convenience, this functionality is also available via the ``vindex[]`` property as has been proposed for numpy.

In [31]:
a = np.arange(15).reshape(5, 3)
za = zarr.array(a, chunks=(3, 2))
za[:]

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [33]:
ix = np.zeros_like(a, dtype=bool)
ix[1, 0] = True
ix[3, 2] = True
za.get_mask_selection(ix)

array([ 3, 11])

In [34]:
za.vindex[ix]

array([ 3, 11])

In [35]:
za.set_mask_selection(ix, 42)
za[:]

array([[ 0,  1,  2],
       [42,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 42],
       [12, 13, 14]])

In [36]:
za.vindex[ix] = 44
za[:]

array([[ 0,  1,  2],
       [44,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 44],
       [12, 13, 14]])

## 1D Benchmarking

In [37]:
c = np.arange(100000000)
c.nbytes

800000000

In [38]:
%time zc = zarr.array(c)
zc.info

CPU times: user 520 ms, sys: 44 ms, total: 564 ms
Wall time: 171 ms


Type,zarr.core.Array
Data type,int64
Shape,"(100000000,)"
Chunk shape,"(97657,)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,builtins.dict
No. bytes,800000000 (762.9M)
No. bytes stored,11854081 (11.3M)
Storage ratio,67.5


In [39]:
%time c.copy()

CPU times: user 116 ms, sys: 60 ms, total: 176 ms
Wall time: 177 ms


array([       0,        1,        2, ..., 99999997, 99999998, 99999999])

In [40]:
%time zc[:]

CPU times: user 492 ms, sys: 80 ms, total: 572 ms
Wall time: 282 ms


array([       0,        1,        2, ..., 99999997, 99999998, 99999999])

### bool dense selection

In [44]:
# relatively dense selection - 10%
ix_dense_bool = np.random.binomial(1, 0.1, size=c.shape[0]).astype(bool)
np.count_nonzero(ix_dense_bool)

9995616

In [45]:
%time c[ix_dense_bool]

CPU times: user 348 ms, sys: 8 ms, total: 356 ms
Wall time: 355 ms


array([      25,       30,       31, ..., 99999973, 99999982, 99999986])

In [46]:
%time zc[ix_dense_bool]

CPU times: user 908 ms, sys: 68 ms, total: 976 ms
Wall time: 474 ms


array([      25,       30,       31, ..., 99999973, 99999982, 99999986])

In [47]:
cProfile.run('zc[ix_dense_bool]', sort='time')

         58428 function calls in 0.492 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1025    0.203    0.000    0.203    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
     1024    0.158    0.000    0.168    0.000 core.py:964(_decode_chunk)
     1024    0.051    0.000    0.240    0.000 core.py:802(_chunk_getitem)
     1024    0.014    0.000    0.014    0.000 {built-in method numpy.core.multiarray.count_nonzero}
     1025    0.008    0.000    0.232    0.000 indexing.py:486(__iter__)
     1024    0.006    0.000    0.214    0.000 index_tricks.py:26(ix_)
     2048    0.006    0.000    0.006    0.000 core.py:324(<genexpr>)
     2048    0.005    0.000    0.005    0.000 {method 'reshape' of 'numpy.ndarray' objects}
     1024    0.004    0.000    0.004    0.000 {built-in method numpy.core.multiarray.frombuffer}
        1    0.004    0.004    0.475    0.475 core.py:563(_get_selection)
     1024    0.003    0.000    0.006 

Method ``nonzero`` is being called internally within numpy to convert bool to int selections, no way to avoid.

### int dense selection

In [48]:
ix_dense_int = np.random.choice(c.shape[0], size=c.shape[0]//10, replace=True)
ix_dense_int_sorted = ix_dense_int.copy()
ix_dense_int_sorted.sort()
len(ix_dense_int)

10000000

In [49]:
%time c[ix_dense_int_sorted]

CPU times: user 60 ms, sys: 4 ms, total: 64 ms
Wall time: 64.1 ms


array([       6,       23,       34, ..., 99999974, 99999986, 99999992])

In [50]:
%time zc[ix_dense_int_sorted]

CPU times: user 560 ms, sys: 100 ms, total: 660 ms
Wall time: 386 ms


array([       6,       23,       34, ..., 99999974, 99999986, 99999992])

In [51]:
%time c[ix_dense_int]

CPU times: user 108 ms, sys: 28 ms, total: 136 ms
Wall time: 135 ms


array([95165047, 93422705,  3887249, ..., 41392662, 20111139, 95001327])

In [52]:
%time zc[ix_dense_int]

CPU times: user 2.11 s, sys: 84 ms, total: 2.19 s
Wall time: 1.86 s


array([95165047, 93422705,  3887249, ..., 41392662, 20111139, 95001327])

In [53]:
cProfile.run('zc[ix_dense_int_sorted]', sort='time')

         55382 function calls in 0.415 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.158    0.158    0.216    0.216 indexing.py:325(__init__)
     1024    0.085    0.000    0.089    0.000 core.py:964(_decode_chunk)
     1024    0.042    0.000    0.145    0.000 core.py:802(_chunk_getitem)
     1025    0.031    0.000    0.031    0.000 indexing.py:372(__iter__)
        1    0.029    0.029    0.029    0.029 {built-in method numpy.core.multiarray.bincount}
        1    0.025    0.025    0.025    0.025 function_base.py:1848(diff)
     1025    0.005    0.000    0.049    0.000 indexing.py:486(__iter__)
     2048    0.004    0.000    0.004    0.000 core.py:324(<genexpr>)
        1    0.003    0.003    0.415    0.415 core.py:527(get_orthogonal_selection)
     1024    0.003    0.000    0.007    0.000 index_tricks.py:26(ix_)
        4    0.003    0.001    0.003    0.001 {method 'reduce' of 'numpy.ufunc' objects}
     2

In [54]:
cProfile.run('zc[ix_dense_int]', sort='time')

         69726 function calls in 1.841 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.160    1.160    1.160    1.160 {method 'argsort' of 'numpy.ndarray' objects}
        1    0.164    0.164    1.490    1.490 indexing.py:325(__init__)
     1024    0.150    0.000    0.285    0.000 core.py:802(_chunk_getitem)
        1    0.128    0.128    0.128    0.128 {method 'take' of 'numpy.ndarray' objects}
     1024    0.113    0.000    0.120    0.000 core.py:964(_decode_chunk)
     1025    0.033    0.000    0.034    0.000 indexing.py:372(__iter__)
        1    0.024    0.024    0.024    0.024 {built-in method numpy.core.multiarray.bincount}
        1    0.012    0.012    0.012    0.012 function_base.py:1848(diff)
     1025    0.006    0.000    0.059    0.000 indexing.py:486(__iter__)
     2048    0.006    0.000    0.012    0.000 index_tricks.py:26(ix_)
     3072    0.004    0.000    0.004    0.000 {method 'reshape' of 

When indices are not sorted, zarr needs to partially sort them so the occur in chunk order, so we only have to visit each chunk once. This sorting dominates the processing time and is unavoidable AFAIK.

### bool sparse selection

In [55]:
# relatively sparse selection
ix_sparse_bool = np.random.binomial(1, 0.0001, size=c.shape[0]).astype(bool)
np.count_nonzero(ix_sparse_bool)

9985

In [56]:
%time c[ix_sparse_bool]

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 21.3 ms


array([    4039,     4499,     7512, ..., 99943621, 99959317, 99987208])

In [57]:
%time zc[ix_sparse_bool]

CPU times: user 436 ms, sys: 56 ms, total: 492 ms
Wall time: 210 ms


array([    4039,     4499,     7512, ..., 99943621, 99959317, 99987208])

In [58]:
cProfile.run('zc[ix_sparse_bool]', sort='time')

         58373 function calls in 0.243 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1023    0.140    0.000    0.148    0.000 core.py:964(_decode_chunk)
     1024    0.024    0.000    0.024    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
     1024    0.009    0.000    0.009    0.000 {built-in method numpy.core.multiarray.count_nonzero}
     1023    0.007    0.000    0.177    0.000 core.py:802(_chunk_getitem)
     1024    0.007    0.000    0.052    0.000 indexing.py:486(__iter__)
     1023    0.006    0.000    0.034    0.000 index_tricks.py:26(ix_)
     2046    0.006    0.000    0.006    0.000 core.py:324(<genexpr>)
     2046    0.004    0.000    0.004    0.000 {method 'reshape' of 'numpy.ndarray' objects}
     1023    0.004    0.000    0.004    0.000 {built-in method numpy.core.multiarray.frombuffer}
        1    0.003    0.003    0.232    0.232 core.py:563(_get_selection)
     1023    0.003    0.000    0.006 

### int sparse selection

In [60]:
ix_sparse_int = np.random.choice(c.shape[0], size=c.shape[0]//10000, replace=True)
ix_sparse_int_sorted = ix_sparse_int.copy()
ix_sparse_int_sorted.sort()
len(ix_sparse_int)

10000

In [61]:
%time c[ix_sparse_int_sorted]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 136 µs


array([    7736,    25765,    27155, ..., 99982813, 99983779, 99986450])

In [62]:
%time c[ix_sparse_int]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 597 µs


array([11023673, 52339189, 27001951, ..., 37185717,  7541357, 28437835])

In [63]:
%time zc[ix_sparse_int_sorted]

CPU times: user 412 ms, sys: 40 ms, total: 452 ms
Wall time: 171 ms


array([    7736,    25765,    27155, ..., 99982813, 99983779, 99986450])

In [64]:
%time zc[ix_sparse_int]

CPU times: user 384 ms, sys: 64 ms, total: 448 ms
Wall time: 172 ms


array([11023673, 52339189, 27001951, ..., 37185717,  7541357, 28437835])

In [65]:
cProfile.run('zc[ix_sparse_int]', sort='time')

         69726 function calls in 0.218 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1024    0.141    0.000    0.149    0.000 core.py:964(_decode_chunk)
     1025    0.008    0.000    0.038    0.000 indexing.py:486(__iter__)
     2048    0.008    0.000    0.015    0.000 index_tricks.py:26(ix_)
     1024    0.006    0.000    0.176    0.000 core.py:802(_chunk_getitem)
     2048    0.006    0.000    0.006    0.000 core.py:324(<genexpr>)
     3072    0.004    0.000    0.004    0.000 {method 'reshape' of 'numpy.ndarray' objects}
     1025    0.004    0.000    0.005    0.000 indexing.py:372(__iter__)
        1    0.003    0.003    0.217    0.217 core.py:563(_get_selection)
     1024    0.003    0.000    0.006    0.000 arrayprint.py:381(wrapper)
     2048    0.003    0.000    0.020    0.000 indexing.py:398(ix_)
     1024    0.003    0.000    0.008    0.000 core.py:319(_cdata_shape)
     1024    0.003    0.000    0.010    0.

For sparse selections, processing time is dominated by decompression, so we can't do any better.

### sparse bool selection as zarr array

In [66]:
zix_sparse_bool = zarr.array(ix_sparse_bool)
zix_sparse_bool.info

Type,zarr.core.Array
Data type,bool
Shape,"(100000000,)"
Chunk shape,"(390625,)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,builtins.dict
No. bytes,100000000 (95.4M)
No. bytes stored,507490 (495.6K)
Storage ratio,197.0


In [67]:
%time zc[zix_sparse_bool]

CPU times: user 920 ms, sys: 136 ms, total: 1.06 s
Wall time: 503 ms


array([    4039,     4499,     7512, ..., 99943621, 99959317, 99987208])

### slice with step

In [68]:
%time np.array(c[::2])

CPU times: user 68 ms, sys: 24 ms, total: 92 ms
Wall time: 90.4 ms


array([       0,        2,        4, ..., 99999994, 99999996, 99999998])

In [69]:
%time zc[::2]

CPU times: user 1.34 s, sys: 236 ms, total: 1.57 s
Wall time: 1.29 s


array([       0,        2,        4, ..., 99999994, 99999996, 99999998])

In [70]:
%time zc[::10]

CPU times: user 548 ms, sys: 116 ms, total: 664 ms
Wall time: 400 ms


array([       0,       10,       20, ..., 99999970, 99999980, 99999990])

In [71]:
%time zc[::100]

CPU times: user 456 ms, sys: 40 ms, total: 496 ms
Wall time: 214 ms


array([       0,      100,      200, ..., 99999700, 99999800, 99999900])

In [72]:
%time zc[::1000]

CPU times: user 440 ms, sys: 36 ms, total: 476 ms
Wall time: 179 ms


array([       0,     1000,     2000, ..., 99997000, 99998000, 99999000])

In [73]:
cProfile.run('zc[::2]', sort='time')

         55382 function calls in 1.351 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.620    0.620    0.815    0.815 indexing.py:325(__init__)
     1024    0.130    0.000    0.135    0.000 core.py:964(_decode_chunk)
     1025    0.127    0.000    0.128    0.000 indexing.py:372(__iter__)
     1024    0.123    0.000    0.275    0.000 core.py:802(_chunk_getitem)
        1    0.121    0.121    0.121    0.121 {built-in method numpy.core.multiarray.bincount}
        1    0.087    0.087    0.087    0.087 {built-in method numpy.core.multiarray.arange}
        1    0.054    0.054    0.054    0.054 function_base.py:1848(diff)
        1    0.020    0.020    1.350    1.350 core.py:527(get_orthogonal_selection)
        4    0.020    0.005    0.020    0.005 {method 'reduce' of 'numpy.ufunc' objects}
     1025    0.007    0.000    0.149    0.000 indexing.py:486(__iter__)
     2048    0.005    0.000    0.005    0.000 core.p

Here there are various setup operations that need to be done on the integer array, can't see way to avoid ATM.

## 2D Benchmarking

In [74]:
c.shape

(100000000,)

In [75]:
d = c.reshape(-1, 1000)
d.shape

(100000, 1000)

In [76]:
zd = zarr.array(d)
zd.info

Type,zarr.core.Array
Data type,int64
Shape,"(100000, 1000)"
Chunk shape,"(3125, 32)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,builtins.dict
No. bytes,800000000 (762.9M)
No. bytes stored,39228864 (37.4M)
Storage ratio,20.4


### bool orthogonal selection

In [77]:
ix0 = np.random.binomial(1, 0.5, size=d.shape[0]).astype(bool)
ix1 = np.random.binomial(1, 0.5, size=d.shape[1]).astype(bool)

In [78]:
%time d[np.ix_(ix0, ix1)]

CPU times: user 140 ms, sys: 8 ms, total: 148 ms
Wall time: 146 ms


array([[       2,        4,        6, ...,      993,      994,      999],
       [    9002,     9004,     9006, ...,     9993,     9994,     9999],
       [   10002,    10004,    10006, ...,    10993,    10994,    10999],
       ..., 
       [99997002, 99997004, 99997006, ..., 99997993, 99997994, 99997999],
       [99998002, 99998004, 99998006, ..., 99998993, 99998994, 99998999],
       [99999002, 99999004, 99999006, ..., 99999993, 99999994, 99999999]])

In [79]:
%time zd.oindex[ix0, ix1]

CPU times: user 748 ms, sys: 56 ms, total: 804 ms
Wall time: 409 ms


array([[       2,        4,        6, ...,      993,      994,      999],
       [    9002,     9004,     9006, ...,     9993,     9994,     9999],
       [   10002,    10004,    10006, ...,    10993,    10994,    10999],
       ..., 
       [99997002, 99997004, 99997006, ..., 99997993, 99997994, 99997999],
       [99998002, 99998004, 99998006, ..., 99998993, 99998994, 99998999],
       [99999002, 99999004, 99999006, ..., 99999993, 99999994, 99999999]])

### int orthogonal selection

In [80]:
ix0 = np.random.choice(d.shape[0], size=int(d.shape[0] * .5), replace=True)
ix1 = np.random.choice(d.shape[1], size=int(d.shape[1] * .5), replace=True)

In [81]:
%time d[np.ix_(ix0, ix1)]

CPU times: user 196 ms, sys: 24 ms, total: 220 ms
Wall time: 219 ms


array([[90796980, 90796608, 90796172, ..., 90796527, 90796979, 90796445],
       [50263980, 50263608, 50263172, ..., 50263527, 50263979, 50263445],
       [47678980, 47678608, 47678172, ..., 47678527, 47678979, 47678445],
       ..., 
       [34172980, 34172608, 34172172, ..., 34172527, 34172979, 34172445],
       [56793980, 56793608, 56793172, ..., 56793527, 56793979, 56793445],
       [12456980, 12456608, 12456172, ..., 12456527, 12456979, 12456445]])

In [82]:
%time zd.oindex[ix0, ix1]

CPU times: user 1.11 s, sys: 68 ms, total: 1.18 s
Wall time: 604 ms


array([[90796980, 90796608, 90796172, ..., 90796527, 90796979, 90796445],
       [50263980, 50263608, 50263172, ..., 50263527, 50263979, 50263445],
       [47678980, 47678608, 47678172, ..., 47678527, 47678979, 47678445],
       ..., 
       [34172980, 34172608, 34172172, ..., 34172527, 34172979, 34172445],
       [56793980, 56793608, 56793172, ..., 56793527, 56793979, 56793445],
       [12456980, 12456608, 12456172, ..., 12456527, 12456979, 12456445]])

### coordinate (point) selection

In [83]:
n = int(d.size * .1)
ix0 = np.random.choice(d.shape[0], size=n, replace=True)
ix1 = np.random.choice(d.shape[1], size=n, replace=True)
n

10000000

In [84]:
%time d[ix0, ix1]

CPU times: user 256 ms, sys: 12 ms, total: 268 ms
Wall time: 265 ms


array([ 6452573, 65841096, 70323990, ..., 44175624, 34778721, 67807976])

In [85]:
%time zd.vindex[ix0, ix1]

CPU times: user 2.62 s, sys: 116 ms, total: 2.73 s
Wall time: 2.29 s


array([ 6452573, 65841096, 70323990, ..., 44175624, 34778721, 67807976])

In [86]:
cProfile.run('zd.vindex[ix0, ix1]', sort='time')

         48293 function calls in 2.312 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.161    1.161    1.161    1.161 {method 'argsort' of 'numpy.ndarray' objects}
        3    0.275    0.092    0.275    0.092 indexing.py:590(<genexpr>)
        3    0.223    0.074    0.223    0.074 indexing.py:581(<genexpr>)
     1024    0.174    0.000    0.358    0.000 core.py:802(_chunk_getitem)
        1    0.167    0.167    1.914    1.914 indexing.py:547(__init__)
     1024    0.155    0.000    0.164    0.000 core.py:964(_decode_chunk)
        1    0.044    0.044    0.044    0.044 {built-in method numpy.core.multiarray.ravel_multi_index}
        1    0.039    0.039    0.039    0.039 {built-in method numpy.core.multiarray.bincount}
     3072    0.022    0.000    0.022    0.000 indexing.py:625(<genexpr>)
     1025    0.009    0.000    0.034    0.000 indexing.py:607(__iter__)
        6    0.005    0.001    0.005    0.001 {me

Points need to be partially sorted so all points in the same chunk are grouped and processed together. This requires ``argsort`` which dominates time.

## h5py comparison

N.B., not really fair because using slower compressor, but for interest...

In [65]:
import h5py
import tempfile

In [78]:
h5f = h5py.File(tempfile.mktemp(), driver='core', backing_store=False)

In [79]:
hc = h5f.create_dataset('c', data=c, compression='gzip', compression_opts=1, chunks=zc.chunks, shuffle=True)
hc

<HDF5 dataset "c": shape (100000000,), type "<i8">

In [80]:
%time hc[:]

CPU times: user 1.16 s, sys: 172 ms, total: 1.33 s
Wall time: 1.32 s


array([       0,        1,        2, ..., 99999997, 99999998, 99999999])

In [81]:
%time hc[ix_sparse_bool]

CPU times: user 1.11 s, sys: 0 ns, total: 1.11 s
Wall time: 1.11 s


array([    1063,    28396,    37229, ..., 99955875, 99979354, 99995791])

In [82]:
# # this is pathological, takes minutes 
# %time hc[ix_dense_bool]

In [83]:
# this is pretty slow
%time hc[::1000]

CPU times: user 38.3 s, sys: 136 ms, total: 38.4 s
Wall time: 38.1 s


array([       0,     1000,     2000, ..., 99997000, 99998000, 99999000])